<a href="https://colab.research.google.com/github/UKJaagadhep/Data-science-and-machine-learning/blob/main/Grammatical%20Error%20Correction%20with%20T5%20SKELETON/Grammatical_Error_Correction_with_T5_SKELETON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Thi

In [68]:
pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.8 MB/s eta 0:00:00


In [72]:
from datasets import load_dataset
from transformers import T5TokenizerFast, BartTokenizerFast
from transformers import TFAutoModelForSeq2SeqLM, TFBartForConditionalGeneration, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import create_optimizer
import tensorflow as tf
import matplotlib.pyplot as plt
import evaluate
import numpy as np

In [15]:
max_length = 128
batch_size = 64

# **DATA PREPARATION**

In [3]:
#dataset_id = "liweili/c4_200m"
dataset_id = "leslyarun/c4_200m_gec_train100k_test25k"

In [4]:
dataset = load_dataset(dataset_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 25000
    })
})

In [21]:
dataset['train'][0]

{'input': 'Bitcoin is for $7,094 this morning, which CoinDesk says.',
 'output': 'Bitcoin goes for $7,094 this morning, according to CoinDesk.'}

In [54]:
model_id = "t5-small"
tokenizer = T5TokenizerFast.from_pretrained(model_id)

In [24]:
#model_id = "lucadiliello/bart-small"
#tokenizer = BartTokenizerFast.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [55]:
def preprocess(examples):
  inputs = [example for example in examples['input']]
  targets = [example for example in examples['output']]

  model_inputs = tokenizer(inputs, text_target = targets, max_length = max_length, truncation = True)
  return model_inputs

In [56]:
tokenized_dataset = dataset.map(preprocess, batched = True, remove_columns = dataset["train"].column_names)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [57]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 25000
    })
})

In [58]:
tokenized_dataset['train'][0]

{'input_ids': [9310,
  19,
  21,
  11301,
  6,
  4198,
  591,
  48,
  1379,
  6,
  84,
  15589,
  2962,
  7,
  157,
  845,
  5,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [9310,
  1550,
  21,
  11301,
  6,
  4198,
  591,
  48,
  1379,
  6,
  1315,
  12,
  15589,
  2962,
  7,
  157,
  5,
  1]}

In [59]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_id)
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = model, return_tensors = "tf")

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
train_dataset = tokenized_dataset["train"].to_tf_dataset(
    shuffle = True,
    batch_size = batch_size,
    collate_fn = data_collator,
)

In [ ]:
val_dataset = tokenized_dataset["test"].to_tf_dataset(
    shuffle = False,
    batch_size = batch_size,
    collate_fn = data_collator,
)

In [ ]:
for i in val_dataset.take(1):
  print(i)

# **MODELING**

In [60]:
model.summary()

Model: "tft5_for_conditional_generation_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  16449536  
                                                                 
 encoder (TFT5MainLayer)     multiple                  35330816  
                                                                 
 decoder (TFT5MainLayer)     multiple                  41625344  
                                                                 
Total params: 60506624 (230.81 MB)
Trainable params: 60506624 (230.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# **TRAINING**

In [ ]:
num_epochs = 5
num_train_steps = len(train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr = 2e-5,
    num_warmup_steps = 0,
    num_train_steps = num_train_steps,
)

In [ ]:
history = model.fit(
  train_dataset,
  validation_data = val_dataset,
  epochs = num_epochs
)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc = 'upper left')
plt.show()

In [ ]:
model.save_weights('/content/bart_small.h5')

# **EVALUATION**

In [71]:
metric = evaluate.load("sacrebleu")

In [ ]:
all_preds = []
all_labels = []

for batch in val_dataset:
  predictions = model.generate(
      input_ids = batch["input_ids"], attention_mask = batch["attention_mask"]
  )
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens = True)
  labels = batch["labels"].numpy()
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens = True)
  all_preds.extend(decoded_preds)
  all_labels.extend(decoded_labels)

result = metric.compute(predictions = all_preds, references = all_labels)
print(result)

# **TESTING**

In [ ]:
wrong_english = "Have you ever played soccer, under the rain, with your friend's?"

tokenized=tokenizer(
  [wrong_english],
  return_tensors='tf'
)

out = model.generate(**tokenized, max_length = max_length)
output_sequence = tokenizer.decode(out[0], skip_special_tokens = True)
print(output_sequence)

## **TESTING ON "juancavallotti/t5-base-gec" (A T5 MODEL PRETRAINED FR GRAMMATICAL ERROR CORRECTION (GEC) TASKS)**

In [61]:
pretrained_model = TFAutoModelForSeq2SeqLM.from_pretrained("juancavallotti/t5-base-gec", from_pt = True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFT5ForConditionalGeneration: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight', 'lm_head.weight']
- This IS expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFT5ForConditionalGeneration from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [62]:
pretrained_model.summary()

Model: "tft5_for_conditional_generation_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (Embedding)          multiple                  24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  109628544 
                                                                 
 decoder (TFT5MainLayer)     multiple                  137949312 
                                                                 
Total params: 222903552 (850.31 MB)
Trainable params: 222903552 (850.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [63]:
wrong_english = [
    "Dady hav'e eateing her foot",
    "DJ Sorryyouwastedyourmoneytobehere",
    "i used to like to swimming",
    "maybe we should organized a meetin with the people from unesco",
    "when are we goinge to start play football",
    "many a time rain fall in my city",
   ]

tokenized = tokenizer(
  wrong_english,
  padding = "longest",
  max_length = max_length,
  truncation = True,
  return_tensors = 'tf'
)

out = pretrained_model.generate(**tokenized, max_length = 128)
print(out)

tf.Tensor(
[[    0 12919    63     3   342   160  2418     3     5     1     0     0
      0     0     0     0     0]
 [    0  6920 11342     6    25 21133    39   540     3     6     1     0
      0     0     0     0     0]
 [    0    27   261    12   114  5989     3     5     1     0     0     0
      0     0     0     0     0]
 [    0  3836    62   225  7958     3     9  1338    28     8   151    45
    245  3523     3     5     1]
 [    0   366    33    62   352    12   456  1556  3370     1     0     0
      0     0     0     0     0]
 [    0  1404     3     9    97  3412  7250    16    82   690     1     0
      0     0     0     0     0]], shape=(6, 17), dtype=int32)


In [64]:
for i in range(len(wrong_english)):
  print(wrong_english[i] + "------------>" + tokenizer.decode(out[i], skip_special_tokens = True))

Dady hav'e eateing her foot------------>Dady ate her foot.
DJ Sorryyouwastedyourmoneytobehere------------>DJ Sorry, you wasted your money,
i used to like to swimming------------>I used to like swimming.
maybe we should organized a meetin with the people from unesco------------>Maybe we should organize a meeting with the people from unesco.
when are we goinge to start play football------------>When are we going to start playing football
many a time rain fall in my city------------>Many a time rain falls in my city


In [65]:
pretrained_model = AutoModelForSeq2SeqLM.from_pretrained("juancavallotti/t5-base-gec")

In [66]:
wrong_english = [
    "Dady hav'e eateing her foot",
    "DJ Sorryyouwastedyourmoneytobehere",
    "i used to like to swimming",
    "maybe we should organized a meetin with the people from unesco",
    "when are we goinge to start play football",
    "many a time rain fall in my city",
   ]

tokenized = tokenizer(
  wrong_english,
  padding = "longest",
  max_length = max_length,
  truncation = True,
  return_tensors = 'pt'
)

out = pretrained_model.generate(**tokenized, max_length = 128)
print(out)

tensor([[    0, 12919,    63,     3,   342,   160,  2418,     3,     5,     1,
             0,     0,     0,     0,     0,     0,     0],
        [    0,  6920, 11342,     6,    25, 21133,    39,   540,     3,     6,
             1,     0,     0,     0,     0,     0,     0],
        [    0,    27,   261,    12,   114,  5989,     3,     5,     1,     0,
             0,     0,     0,     0,     0,     0,     0],
        [    0,  3836,    62,   225,  7958,     3,     9,  1338,    28,     8,
           151,    45,   245,  3523,     3,     5,     1],
        [    0,   366,    33,    62,   352,    12,   456,  1556,  3370,     1,
             0,     0,     0,     0,     0,     0,     0],
        [    0,  1404,     3,     9,    97,  3412,  7250,    16,    82,   690,
             1,     0,     0,     0,     0,     0,     0]])


In [67]:
for i in range(len(wrong_english)):
  print(wrong_english[i] + "------------>" + tokenizer.decode(out[i], skip_special_tokens = True))

Dady hav'e eateing her foot------------>Dady ate her foot.
DJ Sorryyouwastedyourmoneytobehere------------>DJ Sorry, you wasted your money,
i used to like to swimming------------>I used to like swimming.
maybe we should organized a meetin with the people from unesco------------>Maybe we should organize a meeting with the people from unesco.
when are we goinge to start play football------------>When are we going to start playing football
many a time rain fall in my city------------>Many a time rain falls in my city
